In [3]:
from transformers import GPTNeoXTokenizerFast
tokenizer = GPTNeoXTokenizerFast.from_pretrained("EleutherAI/gpt-neox-20b")

In [3]:
tokenizer

Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


GPTNeoXTokenizerFast(name_or_path='EleutherAI/gpt-neox-20b', vocab_size=50254, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<|padding|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	50254: AddedToken("                        ", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	50255: AddedToken("                       ", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	50256: AddedToken("                      ", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	50257: AddedToken("                     ",

In [17]:
import numpy as np
from tqdm import tqdm
import time
import math
import gc
import torch
from datasets import load_dataset
from transformers import GPTNeoXForCausalLM, GPTNeoXTokenizerFast, DataCollatorForLanguageModeling
from utils import create_dataset

ImportError: cannot import name 'create_chunked_dataset_from_full_sequences' from 'utils' (/Users/alexinf/Dropbox/github/reverse-dynamics-nlp/reverse_llm_benchmarking/utils.py)

In [138]:
from importlib import reload
import utils as u
reload(u)

<module 'utils' from '/Users/alexinf/Dropbox/github/reverse-dynamics-nlp/reverse_llm_benchmarking/utils.py'>

In [139]:
test_dataset = u.create_chunked_dataset_from_full_sequences(
        "DebugDataSet",
        tokenizer,
        1,
        2,
        batch_size=1,
        seed=42,
        return_all = True
)

In [141]:
for (i,batch) in enumerate(test_dataset):  
  print(i, " : ", tokenizer.decode(batch['input_ids'][0]))
  print(batch['input_ids'].shape)

0  :   President Donald Trump
torch.Size([1, 3])
1  :   filed a lawsuit
torch.Size([1, 3])
2  :   against former President
torch.Size([1, 3])
3  :   Barack Obama
torch.Size([1, 2])


In [127]:
dataloader = u.create_dataset(
    "pile_val",
    tokenizer,
    10,
    49,
    suffix_length=1,
    num_buffer=0,
    suffix_batch_size=1,
    seed=42
)

In [117]:
for batch in dataloader:
    print(batch["input_ids"].shape)

torch.Size([1, 50])
torch.Size([1, 50])
torch.Size([1, 50])
torch.Size([1, 50])
torch.Size([1, 50])
torch.Size([1, 50])
torch.Size([1, 50])
torch.Size([1, 50])
torch.Size([1, 50])
torch.Size([1, 50])


In [5]:
num_examples = 20
dataset = load_dataset('json', data_files='data/val.jsonl')
dataset = dataset.shuffle(seed=42)
dataset = dataset['train'].select(range(num_examples))

In [103]:
tokenized_dataset = dataset.map(lambda data: {
    'input_ids': tokenizer.encode(
        data['text'],
        return_tensors='pt'
    ).squeeze(0)
})

Map: 100%|██████████| 20/20 [00:00<00:00, 476.61 examples/s]


In [27]:
type(tokenized_dataset[0]["input_ids"])

list

In [14]:
tokenized_dataset = tokenized_dataset.map(lambda x: {'num_tokens': len(x['input_ids'])})

Map: 100%|██████████| 20/20 [00:00<00:00, 1874.97 examples/s]


In [16]:
dataloader = u.

[326,
 1025,
 118,
 144,
 80,
 309,
 802,
 67,
 146,
 977,
 91,
 211,
 614,
 657,
 694,
 177,
 99,
 148,
 527,
 352]

In [4]:
prefix_length = 10
suffix = " Obama"
tokenized_suffix= tokenizer.encode(suffix, return_tensors="pt").to(device)
# tokenized_suffix = tokenized_suffix.unsqueeze(0)
suffix_length = len(tokenized_suffix[0])
empirical_dist = torch.load("..\data\pythia-160m-deduped-v0_stationary_dist.pt").cuda()

vocab_size = empirical_dist.shape[0]

In [5]:
from reverse_sampling import sample_reverse_dynamics_reverse_prior

output1, logits1 = sample_reverse_dynamics_reverse_prior(
    model,
    reverse_model,
    prefix_length,
    tokenized_suffix,
    vocab_batch_size=128,
    temperature=0.7,
)

100%|██████████| 393/393 [01:04<00:00,  6.08it/s]


In [26]:
priors = torch.load("../data/distributions/probs_10.pt")

In [27]:
priors[priors==0] = 1e-16
priors = priors/priors.sum(dim=0)
torch.min(priors)

tensor(1.0000e-16)

In [29]:
suffix = " President Donald Trump filed a lawsuit against former President Barack Obama"
tokenized_suffix= tokenizer.encode(suffix, return_tensors="pt").to(device)

loss = rs.compute_loss_reverse_dynamics(
    model,
    priors,
    tokenized_suffix,
    vocab_batch_size=128,
)

100%|██████████| 393/393 [00:16<00:00, 24.39it/s]


In [7]:
loss

4.131247520446777

In [3]:
from importlib import reload
import reverse_sampling as rs
reload(rs)

<module 'reverse_sampling' from '/home/adi224/reverse-dynamics-nlp/reverse-llm-benchmarking/reverse_sampling.py'>

In [11]:
suffix = " Hi, my name is Alex. What's up?"
tokenized_suffix= tokenizer.encode(suffix, return_tensors="pt").to(device)

loss = rs.compute_loss_reverse_dynamics_reverse_prior(
    model,
    reverse_model,
    tokenized_suffix,
    vocab_batch_size=2000,
)
print(loss)

100%|██████████| 26/26 [00:14<00:00,  1.85it/s]

4.108232021331787


In [19]:
loss_1B = rs.compute_loss_reverse_dynamics_reverse_prior(
    model_1B,
    reverse_model,
    tokenized_suffix,
    vocab_batch_size=500,
)
print(loss_1B)

100%|██████████| 101/101 [01:29<00:00,  1.12it/s]

4.162289619445801


In [ ]:
suffix = " Hi, my name is Alex. What's up?"
tokenized_suffix= tokenizer.encode(suffix, return_tensors="pt").to(device)

loss = rs.compute_loss_reverse_dynamics_reverse_prior(
    model,
    reverse_model,
    tokenized_suffix,
    vocab_batch_size=2000,
)
print(loss)

In [13]:
reversed_suffix = torch.flip(tokenized_suffix, dims=[1])

reverse_model.eval()
reverse_logits = reverse_model(reversed_suffix[:,:-1]).logits
reverse_loss = torch.nn.CrossEntropyLoss()(reverse_logits[0,:,:], reversed_suffix[0, 1:])
print(reverse_loss)

tensor(2.7155, device='cuda:0', grad_fn=<NllLossBackward0>)


In [14]:
model.eval()
forward_logits = model(tokenized_suffix[:,:-1]).logits
forward_loss = torch.nn.CrossEntropyLoss()(forward_logits[0,:,:], tokenized_suffix[0, 1:])
print(forward_loss)

tensor(2.8015, device='cuda:0', grad_fn=<NllLossBackward0>)


2.780669689178467

In [5]:
from reverse_sampling import compute_loss_reverse_dynamics_reverse_prior


suffix = " President Donald Trump filed a lawsuit against former President Barack Obama"
tokenized_suffix= tokenizer.encode(suffix, return_tensors="pt").to(device)

loss = compute_loss_reverse_dynamics_reverse_prior(
    model,
    reverse_model,
    tokenized_suffix,
    vocab_batch_size=128,
)

100%|██████████| 393/393 [00:16<00:00, 23.94it/s]


In [6]:
print(loss)

0.8440132141113281


In [12]:
tokenized_suffix

tensor([[ 3918, 10053,  3778,  4724,   247, 15091,  1411,  3438,  3918, 22306,
          6729]], device='cuda:0')

In [15]:
%run stationary_reversal_loss.py --num_examples 1 --vocab_batch_size 6288 --reverse_model_prior true --prefix_length 11

Computing loss:   0%|          | 0/1 [00:00<?, ?it/s]You're using a GPTNeoXTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


tensor([[ 3918, 10053,  3778,  4724,   247, 15091,  1411,  3438,  3918, 22306,
          6729]], device='cuda:0')



100%|██████████| 8/8 [00:02<00:00,  3.37it/s]

100%|██████████| 8/8 [00:03<00:00,  2.27it/s]

100%|██████████| 8/8 [00:04<00:00,  1.70it/s]

100%|██████████| 8/8 [00:05<00:00,  1.35it/s]

100%|██████████| 8/8 [00:07<00:00,  1.08it/s]

100%|██████████| 8/8 [00:08<00:00,  1.03s/it]

100%|██████████| 8/8 [00:09<00:00,  1.25s/it]

100%|██████████| 8/8 [00:11<00:00,  1.41s/it]

100%|██████████| 8/8 [00:12<00:00,  1.57s/it]

Computing loss: 100%|██████████| 1/1 [01:23<00:00, 83.94s/it]


In [1]:
from reverse_sampling import compute_loss_reverse_dynamics

suffix = " President Donald Trump filed a lawsuit against former President Barack Obama"
tokenized_suffix= tokenizer.encode(suffix, return_tensors="pt").to(device)

loss = compute_loss_reverse_dynamics(
    model,
    empirical_dist,
    tokenized_suffix,
    dilution=1.0,
    vocab_batch_size=128,
)

In [ ]:
dataloader = create_dateset()

### Check Posterior vs Stationary Reversal

In [5]:
uniform_dist = torch.ones_like(empirical_dist) / empirical_dist.shape[0]
empirical_dist = empirical_dist * 0.7 + uniform_dist * 0.3

In [6]:
from reverse_sampling import sample_reverse_dynamics

output1, logits1 = sample_reverse_dynamics(
    model,
    empirical_dist,
    prefix_length,
    tokenized_suffix,
    temperature=0.7,
    vocab_batch_size=512
)

100%|██████████| 99/99 [00:24<00:00,  3.97it/s]


In [8]:
tokenizer.decode(output1[0])

' In thiserior pair, Obama'

In [10]:
logits2 = sr.stationary_reverse_full_dist_suffix_calculation(model, empirical_dist, output1,)

i= 0


100%|██████████| 32/32 [00:25<00:00,  1.27it/s]


i= 0


100%|██████████| 32/32 [00:25<00:00,  1.27it/s]


i= 0


100%|██████████| 32/32 [00:22<00:00,  1.42it/s]


i= 0


100%|██████████| 32/32 [00:18<00:00,  1.73it/s]


i= 0


100%|██████████| 32/32 [00:14<00:00,  2.25it/s]


In [11]:
logits1.log_softmax(dim=-1)

tensor([[-14.1750, -14.4474, -15.0450,  ..., -13.0240, -12.8964, -13.3337],
        [-12.3171, -12.7534, -13.0377,  ..., -12.1253,  -9.1679, -14.2333],
        [-13.4286, -12.3321, -11.0569,  ..., -10.6190, -11.5005, -12.7546],
        [-11.3032, -11.4353, -13.2914,  ..., -12.1252, -11.5411, -13.5619],
        [-13.5377, -15.0662,  -8.3655,  ..., -14.6411, -13.6813, -13.2714]],
       device='cuda:0')

In [12]:
torch.abs(logits2 - logits1.log_softmax(dim=-1)).max()

tensor(3.0518e-05, device='cuda:0')

In [36]:
import json

In [39]:
import json
import glob

# Step 1: Create an empty list to hold all JSON objects.
all_data = []

# Step 2: Loop through each JSON file in the directory where your files are stored.
for filename in glob.glob('data/pile_val/*.json'):
    with open(filename, 'r') as file:
        # Read the single line of JSON and parse it.
        json_data = json.loads(file.readline())
        # Append the dictionary to the list.
        all_data.append(json_data)

# Step 3: Write the list of dictionaries to a new JSON file.
with open('combined_data.json', 'w') as outfile:
    json.dump(all_data, outfile)

In [43]:
all_data

[{'mean': 5.902055968046188,
  'variance': 2.9243932490525784,
  'std_on_mean': 0.17100857431873345,
  'nbatches': 100,
  'num_examples': 100,
  'prefix_length': 10,
  'suffix_length': 1,
  'num_buffer': 0,
  'suffix_batch_size': 1,
  'vocab_batch_size': 12576,
  'device': 'cuda',
  'dist': '../data/distributions/pile10k_empirical.pt',
  'dilution': 0.2},
 {'mean': 3.7944451820850373,
  'variance': 1.3930675834793667,
  'std_on_mean': 0.11802828404578991,
  'total_samples': 100,
  'batch_size': 1,
  'nbatches': 100,
  'sample_length': 2048},
 {'mean': 5.567424488067627,
  'variance': 2.3046989259260138,
  'std_on_mean': 0.48007279926340474,
  'nbatches': 10,
  'num_examples': 10,
  'prefix_length': 10,
  'suffix_length': 1,
  'num_buffer': 0,
  'suffix_batch_size': 1,
  'vocab_batch_size': 6288,
  'device': 'cuda',
  'dist': None,
  'dilution': 0.0,
  'reverse_model_prior': True,
  'multiple_priors_start_idx': 0,
  'multiple_priors_end_idx': 0,
  'model_size': '410m'},
 {'mean': 5.8819